# LightGBM Model Selection

In [6]:
from lightgbm import LGBMRegressor # The ML model

from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import wandb
from sklearn.pipeline import FunctionTransformer


from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.methods import MinTrace, BottomUp
from hierarchicalforecast.utils import aggregate
from hierarchicalforecast.core import HierarchicalReconciliation
from mlforecast import MLForecast
from mlforecast.target_transforms import GlobalSklearnTransformer

from valuation.infra.store.dataset import DatasetStore
from valuation.asset.identity.dataset import DatasetID
from valuation.core.stage import DatasetStage
from valuation.asset.identity.model import ModelPassport
from valuation.asset.model.mlforecast import MLForecastModel
from valuation.infra.store.model import ModelStore
from valuation.flow.modeling.model_selection.light_gbm import LightGBMHP
from valuation.flow.modeling.model_selection.cv import CrossValidationHP
from valuation.flow.modeling.model_selection.mlforecast import MLForecastHP
from valuation.flow.modeling.model_selection.base import ModelParams

# Suppress the specific warning
warnings.filterwarnings('ignore', message='.*lag.*')
warnings.filterwarnings('ignore', message='.*UnsupportedFieldAttributeWarning.*')
wandb.login()

True

## Model Parameters

In [7]:
NUM_CORES = 24              # Total CPU cores available
# Reproducibility 
RANDOM_STATE = 42           # Fixed random seed

MODELS = [LGBMRegressor(**LightGBMHP(tweedie_variance_power=1.2).as_dict()),
          LGBMRegressor(**LightGBMHP(tweedie_variance_power=1.3).as_dict()),
          LGBMRegressor(**LightGBMHP(tweedie_variance_power=1.4).as_dict()),
          LGBMRegressor(**LightGBMHP(tweedie_variance_power=1.5).as_dict()),
]


## Training Data

In [8]:
store = DatasetStore()
dataset_id = DatasetID(name="train_val", stage=DatasetStage.MODEL)
passport = store.get_passport(dataset_id=dataset_id)
ds = store.get(passport=passport)
train_df = ds.data

2025-10-24 03:52:19.772 | DEBUG    | valuation.asset.dataset.base:load:338 - Dataset Dataset train_val of the model stage created on 2025-10-24 at 01:30 loaded.


## Feature Engineering

In [9]:
mf = MLForecast(models=MODELS,**MLForecastHP().as_dict())

## Blocked Cross-Validation
This generates the unreconciled forecasts for each fold. We must add fitted=True to get the in-sample forecasts for the reconciler.

In [10]:
cv_df_base = mf.cross_validation(
    df=train_df, **CrossValidationHP().as_dict())
mf.fit(df=train_df)

MLForecast(models=[LGBMRegressor, LGBMRegressor2, LGBMRegressor3, LGBMRegressor4], freq=W-WED, lag_features=['lag1', 'lag2', 'lag4', 'lag13', 'lag52', 'lag43', 'rolling_mean_lag1_window_size4', 'rolling_mean_lag1_window_size13', 'rolling_mean_lag52_window_size5'], date_features=['week', 'month', 'dayofyear'], num_threads=22)

## Create Summing Matrix and Tags

In [11]:
# 1. Start with the core data (unique_id, ds, y)
hierarchy_df = train_df[['unique_id', 'ds', 'y']].drop_duplicates() 

# 2. Create the grouping columns
hierarchy_df['store'] = hierarchy_df['unique_id'].apply(lambda s: s.split('_')[0])
hierarchy_df['category'] = hierarchy_df['unique_id'].apply(lambda s: s.split('_')[1])

# 3. Drop the 'unique_id' column from the input DF before aggregation
#    The aggregate function knows to use the combination of columns in 'spec'
#    to uniquely identify the time series levels.
hierarchy_df_clean = hierarchy_df.drop(columns=['unique_id']) # 👈 ADD THIS LINE

spec = [['store'], ['category'], ['store', 'category']]

# Pass the cleaned DataFrame to the aggregate function
_, S_df, tags = aggregate(df=hierarchy_df_clean, spec=spec)

## Aggregate Forecasts

In [12]:
# Clean and Prepare Y_hat_df_base
Y_hat_df_base = cv_df_base.drop(columns=['cutoff', 'y'], errors='ignore')

# 1. Add the hierarchy columns to the base forecasts
Y_hat_df_base_clean = Y_hat_df_base.copy()
Y_hat_df_base_clean['store'] = Y_hat_df_base_clean['unique_id'].apply(lambda s: s.split('_')[0])
Y_hat_df_base_clean['category'] = Y_hat_df_base_clean['unique_id'].apply(lambda s: s.split('_')[1])

# 2. Identify the forecast column(s) - typically 'LGBMRegressor' or similar model name
forecast_col = 'LGBMRegressor'  # Adjust if your column has a different name

# 3. Rename forecast column to 'y' temporarily for aggregation
Y_hat_df_base_for_agg = Y_hat_df_base_clean.copy()
Y_hat_df_base_for_agg = Y_hat_df_base_for_agg.rename(columns={forecast_col: 'y'})
Y_hat_df_base_for_agg = Y_hat_df_base_for_agg.drop(columns=['unique_id'])

# 4. Aggregate to create forecasts at all hierarchy levels
Y_hat_aggregated, _, _ = aggregate(df=Y_hat_df_base_for_agg, spec=spec)

# 5. Rename back to original forecast column name
Y_hat_aggregated = Y_hat_aggregated.rename(columns={'y': forecast_col})

print("Forecasts aggregated successfully across all hierarchy levels.")
print(f"Base forecasts shape: {Y_hat_df_base.shape}")
print(f"Aggregated forecasts shape: {Y_hat_aggregated.shape}")
print(f"Unique IDs in aggregated forecasts: {Y_hat_aggregated['unique_id'].nunique()}")


Forecasts aggregated successfully across all hierarchy levels.
Base forecasts shape: (202800, 6)
Aggregated forecasts shape: (141492, 3)
Unique IDs in aggregated forecasts: 2721


## Reconciler for CV Forecasts

In [13]:
reconcilers = [MinTrace(method='ols')]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)

# Prepare aggregated actuals
Y_df_base = cv_df_base[['unique_id', 'ds', 'y']].copy()
Y_df_base['store'] = Y_df_base['unique_id'].apply(lambda s: s.split('_')[0])
Y_df_base['category'] = Y_df_base['unique_id'].apply(lambda s: s.split('_')[1])
Y_df_base_for_agg = Y_df_base.drop(columns=['unique_id'])
Y_df_actuals, _, _ = aggregate(df=Y_df_base_for_agg, spec=spec)

# Reconcile (this adjusts the aggregated forecasts for coherence)
cv_df_reconciled = hrec.reconcile(
    Y_hat_df=Y_hat_aggregated,  # Now has all hierarchy levels
    Y_df=Y_df_actuals,
    S_df=S_df,
    tags=tags
)

## Evaluate CV Performance

In [14]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import numpy as np

from valuation.utils.metrics import compute_smape, compute_wape

run = wandb.init(project="valuation", job_type="modeling")

# 1. Get actuals at ALL hierarchy levels
actuals_base = cv_df_base[['unique_id', 'ds', 'cutoff', 'y']].copy()
actuals_base['store'] = actuals_base['unique_id'].apply(lambda s: s.split('_')[0])
actuals_base['category'] = actuals_base['unique_id'].apply(lambda s: s.split('_')[1])
actuals_base_for_agg = actuals_base.drop(columns=['unique_id'])

# Aggregate actuals
actuals_aggregated, _, _ = aggregate(df=actuals_base_for_agg, spec=spec)

# 2. Merge forecasts with actuals
cv_df_eval = cv_df_reconciled.merge(
    actuals_aggregated[['unique_id', 'ds', 'y']], 
    on=['unique_id', 'ds'],
    how='left'
)

# 3. Classify hierarchy levels properly
def classify_level(uid):
    if '_' in uid:
        return 'bottom'  # store_category (e.g., "100_beer")
    elif '/' in uid:
        return 'store_category'  # aggregated store/category (e.g., "100/beer")
    else:
        # Check if it's a store (numeric) or category (text)
        try:
            int(uid)
            return 'store'  # Just store (e.g., "100")
        except:
            return 'category'  # Just category (e.g., "beer")

cv_df_eval['level'] = cv_df_eval['unique_id'].apply(classify_level)

# 4. Get model columns
model_cols = [col for col in cv_df_reconciled.columns 
              if col not in ['unique_id', 'ds', 'cutoff']]

print(f"Found model columns: {model_cols}")
print(f"\nDataset overview:")
print(f"  Total forecasts: {len(cv_df_eval):,}")
print(f"  Unique series: {cv_df_eval['unique_id'].nunique():,}")
print(f"  Date range: {cv_df_eval['ds'].min()} to {cv_df_eval['ds'].max()}")
print(f"\nActual values summary:")
print(cv_df_eval['y'].describe())

# 5. Overall Performance
print("\n" + "="*80)
print("OVERALL PERFORMANCE (All Hierarchy Levels)")
print("="*80)

performance_results = []
for model_col in model_cols:
    mask = cv_df_eval[[model_col, 'y']].notna().all(axis=1)
    y_true = cv_df_eval.loc[mask, 'y']
    y_pred = cv_df_eval.loc[mask, model_col]
    
    performance = {
        'model': model_col.replace('LGBMRegressor/', ''),  # Shorter names
        'MSE': mean_squared_error(y_true, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
        'MAE': mean_absolute_error(y_true, y_pred),
        'MAPE': mean_absolute_percentage_error(y_true, y_pred) * 100,  # As percentage
        'SMAPE': compute_smape(y_true, y_pred),
        'WAPE': compute_wape(y_true, y_pred),
        'Mean_Actual': y_true.mean(),
        'n_forecasts': len(y_true)}
    
    performance_results.append(performance)
    run.log(performance)

overall_perf = pd.DataFrame(performance_results).set_index('model')
print(overall_perf.to_string())

# 6. Performance by Hierarchy Level
print("\n" + "="*80)
print("PERFORMANCE BY HIERARCHY LEVEL")
print("="*80)

levels_order = ['bottom', 'store_category', 'store', 'category']
level_results = []

for level in levels_order:
    level_data = cv_df_eval[cv_df_eval['level'] == level]
    
    if len(level_data) == 0:
        continue
        
    print(f"\n{level.upper()} Level:")
    print(f"  Unique series: {level_data['unique_id'].nunique():,}")
    print(f"  Total forecasts: {len(level_data):,}")
    print(f"  Actual mean: {level_data['y'].mean():.2f}, std: {level_data['y'].std():.2f}")
    
    for model_col in model_cols:
        mask = level_data[[model_col, 'y']].notna().all(axis=1)
        y_true = level_data.loc[mask, 'y']
        y_pred = level_data.loc[mask, model_col]
        
        if len(y_true) > 0:

            rmse_val = np.sqrt(mean_squared_error(y_true, y_pred))
            mae_val = mean_absolute_error(y_true, y_pred)
            mape_val = mean_absolute_percentage_error(y_true, y_pred) * 100
            smape_val = compute_smape(y_true, y_pred)
            wape_val = compute_wape(y_true, y_pred)
            
            performance = {
                "Level": level,
                "Model": model_col.replace('LGBMRegressor/', ''),
                'RMSE': rmse_val,
                'MAE': mae_val,
                'MAPE%': mape_val,
                'Mean_Actual': y_true.mean(),
                'n': len(y_true)
            }
            
            level_results.append(performance)
            
            # Normalized error (MAE as % of mean)
            normalized_mae = (mae_val / y_true.mean() * 100) if y_true.mean() > 0 else 0
            
            print(f"    {model_col.replace('LGBMRegressor/', '')[:30]:30s} -> "
                  f"RMSE: {rmse_val:>10.2f}, MAE: {mae_val:>10.2f}, "
                  f"MAPE: {mape_val:>6.2f}%, Norm_MAE: {normalized_mae:>6.2f}%")

# 7. Comparison Table
print("\n" + "="*80)
print("RECONCILIATION IMPACT (Comparing Base vs Reconciled)")
print("="*80)

level_perf_df = pd.DataFrame(level_results)
if len(level_perf_df) > 0:
    comparison = level_perf_df.pivot_table(
        index='Level',
        columns='Model',
        values=['RMSE', 'MAE', 'MAPE%']
    )
    print(comparison.to_string())

# 8. Sample predictions vs actuals
print("\n" + "="*80)
print("SAMPLE PREDICTIONS (First 10 bottom-level forecasts)")
print("="*80)

sample = cv_df_eval[cv_df_eval['level'] == 'bottom'].head(10)[
    ['unique_id', 'ds', 'y'] + model_cols
].round(2)
print(sample.to_string())

Found model columns: ['LGBMRegressor', 'LGBMRegressor/MinTrace_method-ols']

Dataset overview:
  Total forecasts: 141,492
  Unique series: 2,721
  Date range: 1995-01-04 00:00:00 to 1995-12-27 00:00:00

Actual values summary:
count         141492.0
mean       30142.27635
std      190899.049374
min                0.0
25%          1334.1525
50%            5400.57
75%           12730.56
max        14800224.78
Name: y, dtype: double[pyarrow]

OVERALL PERFORMANCE (All Hierarchy Levels)
                              MSE          RMSE          MAE          MAPE      SMAPE       WAPE  Mean_Actual  n_forecasts
model                                                                                                                     
LGBMRegressor        4.515441e+09  67197.032441  7446.068713  8.666328e+20  59.512188  24.703074  30142.27635       141492
MinTrace_method-ols  4.515441e+09  67197.032441  7446.068713  8.666328e+20  59.512188  24.703074  30142.27635       141492

PERFORMANCE BY HIERAR

## Model Performance Diagnosis

In [15]:

print("="*80)
print("PERFORMANCE DIAGNOSTICS")
print("="*80)

# 1. Check imputation impact
print("\n1. IMPUTATION ANALYSIS")
print("-"*80)

# Check what percentage of data was originally missing
# This assumes train_df is your final densified/imputed dataset
if 'train_df' in locals():
    # Check series lengths - all should be equal after densification
    series_lengths = train_df.groupby('unique_id').size()
    print(f"Series lengths after densification:")
    print(f"  Min: {series_lengths.min()}, Max: {series_lengths.max()}")
    print(f"  All equal: {len(series_lengths.unique()) == 1}")
    
    # If you tracked NaNs before imputation, report it
    # Otherwise, skip this section
    print("\n⚠️  Note: Imputation percentage not tracked.")
    print("   If performance is poor, imputation may be the cause.")

# 2. Check training data quality
print("\n2. TRAINING DATA QUALITY")
print("-"*80)

if 'train_df' in locals():
    print(f"Training data shape: {train_df.shape}")
    print(f"Unique series: {train_df['unique_id'].nunique()}")
    print(f"Date range: {train_df['ds'].min()} to {train_df['ds'].max()}")
    
    # Check for zero/near-zero values
    zero_pct = (train_df['y'] == 0).sum() / len(train_df) * 100
    near_zero_pct = (train_df['y'] < 1).sum() / len(train_df) * 100
    print(f"\nZero values: {zero_pct:.1f}%")
    print(f"Near-zero (<1): {near_zero_pct:.1f}%")
    
    # Revenue distribution
    print(f"\nRevenue distribution:")
    print(train_df['y'].describe())
    
    # Check for negative values
    if (train_df['y'] < 0).any():
        print(f"⚠️  WARNING: {(train_df['y'] < 0).sum()} negative values found!")

# 3. Check predictions quality
print("\n3. PREDICTION QUALITY ANALYSIS")
print("-"*80)

if 'cv_df_eval' in locals() and 'model_cols' in locals():
    # Check for extreme predictions
    for model_col in model_cols:
        preds = cv_df_eval[model_col].dropna()
        actuals = cv_df_eval['y'].dropna()
        
        print(f"\n{model_col}:")
        print(f"  Predictions range: [{preds.min():.2f}, {preds.max():.2f}]")
        print(f"  Actuals range: [{actuals.min():.2f}, {actuals.max():.2f}]")
        print(f"  Predictions mean: {preds.mean():.2f} vs Actuals mean: {actuals.mean():.2f}")
        
        # Check for negative predictions
        neg_preds = (preds < 0).sum()
        if neg_preds > 0:
            print(f"  ⚠️  {neg_preds} negative predictions ({neg_preds/len(preds)*100:.1f}%)")
        
        # Check for extreme predictions
        extreme_high = (preds > actuals.quantile(0.99) * 2).sum()
        if extreme_high > 0:
            print(f"  ⚠️  {extreme_high} extremely high predictions")

# 4. Residual analysis
print("\n4. RESIDUAL ANALYSIS")
print("-"*80)

if 'cv_df_eval' in locals() and len(model_cols) > 0:
    model_col = model_cols[0]  # Analyze first model
    
    mask = cv_df_eval[[model_col, 'y']].notna().all(axis=1)
    actuals = cv_df_eval.loc[mask, 'y']
    preds = cv_df_eval.loc[mask, model_col]
    residuals = actuals - preds
    
    print(f"Analyzing {model_col}:")
    print(f"  Mean residual: {residuals.mean():.2f}")
    print(f"  Median residual: {residuals.median():.2f}")
    print(f"  Residual std: {residuals.std():.2f}")
    print(f"  Mean absolute error: {np.abs(residuals).mean():.2f}")
    
    # Check for systematic bias
    if abs(residuals.mean()) > actuals.std() * 0.1:
        print(f"  ⚠️  WARNING: Systematic bias detected!")
        if residuals.mean() > 0:
            print(f"     Model is UNDER-predicting on average")
        else:
            print(f"     Model is OVER-predicting on average")

# 5. Check differences transformation
print("\n5. DIFFERENCING IMPACT")
print("-"*80)

if 'TARGET_TRANSFORMS' in locals() and len(MLForecastHP().target_transforms) > 0:
    print(f"Target transforms applied: {MLForecastHP().target_transforms}")
    print("⚠️  Differencing can cause issues if:")
    print("   - Series are short")
    print("   - Series have many imputed values")
    print("   - Series are already stationary")
    print("\n💡 RECOMMENDATION: Try removing Differences([1]) and see if performance improves")

# 6. Check for data leakage
print("\n6. DATA LEAKAGE CHECK")
print("-"*80)

if 'cv_df_base' in locals():
    # Check if CV is working correctly
    cv_windows = cv_df_base.groupby(['unique_id', 'cutoff']).size()
    print(f"CV windows per series: {cv_windows.groupby(level=0).size().value_counts()}")
    
    # Check cutoff dates
    print(f"\nCutoff dates: {sorted(cv_df_base['cutoff'].unique())}")

# 7. Specific problematic series
print("\n7. WORST PERFORMING SERIES")
print("-"*80)

if 'cv_df_eval' in locals() and 'model_cols' in locals():
    model_col = model_cols[0]
    
    # Calculate MAE per series
    series_mae = cv_df_eval.groupby('unique_id').apply(
        lambda x: mean_absolute_error(x['y'].dropna(), x[model_col].dropna()) 
        if len(x['y'].dropna()) > 0 else np.nan
    ).sort_values(ascending=False)
    
    print("Top 10 worst series (by MAE):")
    for uid, mae in series_mae.head(10).items():
        series_data = cv_df_eval[cv_df_eval['unique_id'] == uid]
        actual_mean = series_data['y'].mean()
        pred_mean = series_data[model_col].mean()
        print(f"  {uid:30s} MAE: {mae:>10.2f}, Actual mean: {actual_mean:>10.2f}, Pred mean: {pred_mean:>10.2f}")

print("\n" + "="*80)
print("RECOMMENDATIONS")
print("="*80)
print("""
Based on diagnostics above, try these fixes:

1. REMOVE DIFFERENCING if series are short or heavily imputed:
   TARGET_TRANSFORMS = []  # Remove Differences([1])

2. REDUCE LAGS if many series are short:
   LAGS = [1, 2, 4, 8, 13]  # Remove 26, 52

3. FILTER OUT heavily imputed series (>30% imputed)

4. CHECK if forward/backward fill is creating unrealistic patterns

5. INCREASE min_child_samples for more regularization:
   min_child_samples=50 or 100

6. ADD more regularization:
   reg_alpha=1.0, reg_lambda=1.0
""")

PERFORMANCE DIAGNOSTICS

1. IMPUTATION ANALYSIS
--------------------------------------------------------------------------------
Series lengths after densification:
  Min: 313, Max: 313
  All equal: True

⚠️  Note: Imputation percentage not tracked.
   If performance is poor, imputation may be the cause.

2. TRAINING DATA QUALITY
--------------------------------------------------------------------------------
Training data shape: (813800, 3)
Unique series: 2600
Date range: 1990-01-03 00:00:00 to 1995-12-27 00:00:00

Zero values: 21.5%
Near-zero (<1): 21.5%

Revenue distribution:
count        813800.0
mean      6942.470871
std      10024.185717
min               0.0
25%          836.4825
50%          3891.705
75%         8409.0675
max         950516.31
Name: y, dtype: double[pyarrow]

3. PREDICTION QUALITY ANALYSIS
--------------------------------------------------------------------------------

LGBMRegressor:
  Predictions range: [-14627.85, 10772970.60]
  Actuals range: [0.00, 1480022

/tmp/ipykernel_74513/4220664284.py:129: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  series_mae = cv_df_eval.groupby('unique_id').apply(
